In [7]:
import numpy as np
import pandas as pd

"""
# Exponential Moving Average
# https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages

Params: 
    data: pandas DataFrame
    period: smoothing period
    column: the name of the column with values for calculating EMA in the 'data' DataFrame
    
Returns:
    copy of 'data' DataFrame with 'ema[period]' column added
"""

